In [1]:
import json
import cv2
import numpy as np

from dataclasses import dataclass

from PIL import Image
from IPython import display

import os
import shutil

In [2]:
@dataclass
class Joint:
    x: float
    y: float
    confidence: float
    
    def is_empty(self):
        return self.x == 0 and self.y == 0 and self.confidence == 0

In [3]:
joint_colours = np.linspace(0, 179, 26)[:-1]
joint_colours = np.array([(round(h), 255, 255) for h in joint_colours]).astype(np.uint8)
joint_colours = joint_colours[np.newaxis, ...]
joint_colours = cv2.cvtColor(joint_colours, cv2.COLOR_HSV2BGR)
joint_colours = joint_colours.squeeze().astype(int)
joint_colours = [tuple([int(x) for x in joint_colour]) for joint_colour in joint_colours]

In [8]:
# https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/doc/02_output.md

class PoseClassification:
    INVALID: str = "Invalid"
    HEAD_ONLY: str = "Head Only"
    HALF_BODY: str = "Half Body"
    FULL_BODY_STANDING: str = "Full Body Standing"
    FULL_BODY_SITTING: str = "Full Body Sitting"
    OTHER: str = "Other"
    EMPTY: str = "Empty"
    Y_OTHER: str = "Y Other"
    
@dataclass
class PoseKeypoints:
    nose: Joint
    neck: Joint
    right_shoulder: Joint
    right_elbow: Joint
    right_wrist: Joint
    left_shoulder: Joint
    left_elbow: Joint
    left_wrist: Joint
    mid_hip: Joint
    right_hip: Joint
    right_knee: Joint
    right_ankle: Joint
    left_hip: Joint
    left_knee: Joint
    left_ankle: Joint
    right_eye: Joint
    left_eye: Joint
    right_ear: Joint
    left_ear: Joint
    left_big_toe: Joint
    left_small_toe: Joint
    left_heel: Joint
    right_big_toe: Joint
    right_small_toe: Joint
    right_heel: Joint
    
    def get_empty_points(self):
        return {joint_name: getattr(self, joint_name) for joint_name in self.__annotations__.keys() if getattr(self, joint_name).is_empty()}
    
    def overlay_pose(self, frame):
        mapping = list(PoseKeypoints.__dict__["__annotations__"].keys())
        pose_overlayed_frame = frame.copy()
        height, width, channels = pose_overlayed_frame.shape

        for i, joint_name in enumerate(self.__annotations__.keys()):
            joint = getattr(self, joint_name)

            if joint.is_empty(): 
                continue

            cv2.circle(pose_overlayed_frame, (round(joint.x * width), round(joint.y * height)), radius=1, color=joint_colours[i], thickness=2)
            cv2.putText(pose_overlayed_frame, mapping[i], (round(joint.x * width), round(joint.y * (height - 0.02))), cv2.FONT_HERSHEY_SIMPLEX, 0.5, joint_colours[i], 1, cv2.LINE_AA)
        
        return pose_overlayed_frame

    def classify(self):
        # Note that values close to the top of the image are nearer to 0 and those at the bottom are nearer to 1
        empties = self.get_empty_points()

        # Check for disformed pose (i.e. without enough info to classify)
        if len(empties) > 20:
            return PoseClassification.EMPTY
        
        # Check that they are upright
        mid_hip_y = -self.mid_hip.y
        nose_y = -self.nose.y
        neck_y = -self.neck.y

        if (nose_y < neck_y and nose_y != 0 and neck_y != 0) or (neck_y < mid_hip_y and mid_hip_y != 0 and neck_y != 0):
            return PoseClassification.Y_OTHER
        
        # Check for full body
        # Potentially allow for less to be visible like the bottom of the feet?
        
        empty_key_set = set(empties.keys())
        allowed_empty_keys = set(["left_big_toe", "left_small_toe", "left_heel", "right_big_toe", "right_small_toe", "right_heel"])
        feet_excluded_empties = empty_key_set - allowed_empty_keys
        
        print(feet_excluded_empties)
        
        if len(empties) < 2 or len(feet_excluded_empties) == 0:
            right_hip_y = -self.right_hip.y
            left_hip_y = -self.left_hip.y
            right_knee_y = -self.right_knee.y
            left_knee_y = -self.left_knee.y
            right_ankle_y = -self.right_ankle.y
            left_ankle_y = -self.left_ankle.y
            
            right_ratio = (right_hip_y - right_knee_y) / (right_knee_y - right_ankle_y)
            left_ratio = (left_hip_y - left_knee_y) / (left_knee_y - left_ankle_y)
            
            print(left_ratio, right_ratio)
            
            left_diff = left_hip_y - left_knee_y
            right_diff = right_hip_y - left_knee_y
            
            # For a standing person, left_hip_y > left_knee_y <=> left_diff > 0
            # For a sitting person left_hip_y ~= or < left_knee_y so left_diff ~= or < 0
            # These values use normalised distances making this a bit simpler
            
            print(left_diff, right_diff)
            
            # if left_diff < 0.05 and right_diff < 0.05:
            #     # Approx equal or negative so probably sitting
            #     return PoseClassification.FULL_BODY_SITTING
            
            if left_ratio < 0.65 or right_ratio < 0.65:
                return PoseClassification.FULL_BODY_SITTING
            
            if left_diff > 0.05 and right_diff > 0.05:
                # Both in normal positions
                return PoseClassification.FULL_BODY_STANDING
            
            # Other
            return PoseClassification.OTHER
    
        if not self.right_shoulder.is_empty() and not self.neck.is_empty() and not self.left_shoulder.is_empty() and not self.right_elbow.is_empty() and not self.left_elbow.is_empty():
            return PoseClassification.HALF_BODY
        
        if not self.right_ear.is_empty() and not self.right_eye.is_empty() and not self.nose.is_empty() and not self.left_eye.is_empty() and not self.left_ear.is_empty():
            return PoseClassification.HEAD_ONLY
                
        return PoseClassification.INVALID
    
    @staticmethod
    def load_keypoints(raw_kps):
        mapping = list(PoseKeypoints.__dict__["__annotations__"].keys())
        points_as_dict = {part: Joint(raw_kps[3 * i], raw_kps[3 * i + 1], raw_kps[3 * i + 2]) for i, part in enumerate(mapping)}
        return PoseKeypoints(**points_as_dict)

In [9]:
in_frame_folder = "./output/segmented/Train/Game"
in_pose_folder = "./output/segmented/Train/Game/openpose_out_normalised"
out_folder = "./output/pose_classified"

In [10]:
file_names = []

for file in os.listdir(in_frame_folder):
    path = f"{in_frame_folder}/{file}"
    
    if not os.path.isfile(path):
        continue
        
    file_names.append(''.join(file.split(".")[:-1]))

len(file_names)

135

In [11]:
for file in file_names:
    frame_loc = f"{in_frame_folder}/{file}.jpg"
    keypoints_loc = f"{in_pose_folder}/{file}_keypoints.json"
    
    with open(keypoints_loc, "r") as fp:
        raw_data = json.load(fp)
    
    if len(raw_data["people"]) != 1:
        # print(frame_loc)
        continue
    
    print(frame_loc)
    
    kps = raw_data["people"][0]["pose_keypoints_2d"]
    pose = PoseKeypoints.load_keypoints(kps)
    
    classification = pose.classify()
    save_loc = f"{out_folder}/{classification}/{file}.jpg"
    
    in_frame = cv2.imread(frame_loc)
    out_frame = pose.overlay_pose(in_frame)
    
    print(classification)
    
    cv2.imwrite(save_loc, out_frame)
    

./output/segmented/Train/Game/Segmented_Train_Game_01_00001_0.jpg
set()
0.8257854615402881 1.0739826496820466
0.208638 0.21443
Full Body Standing
./output/segmented/Train/Game/Segmented_Train_Game_01_00001_1.jpg
{'left_ear', 'left_ankle', 'left_knee', 'right_knee', 'right_ankle'}
Half Body
./output/segmented/Train/Game/Segmented_Train_Game_01_00009_0.jpg
{'mid_hip', 'left_ankle', 'right_elbow', 'left_knee', 'right_wrist', 'left_wrist', 'right_knee', 'left_hip', 'right_ankle', 'right_hip'}
Head Only
./output/segmented/Train/Game/Segmented_Train_Game_01_00017_0.jpg
{'mid_hip', 'left_ankle', 'right_elbow', 'left_knee', 'right_wrist', 'left_wrist', 'left_elbow', 'left_hip', 'right_knee', 'right_ankle', 'right_hip'}
Head Only
./output/segmented/Train/Game/Segmented_Train_Game_01_00025_0.jpg
{'mid_hip', 'left_ankle', 'right_elbow', 'left_knee', 'right_wrist', 'left_wrist', 'right_knee', 'left_hip', 'right_ankle', 'right_hip'}
Head Only
./output/segmented/Train/Game/Segmented_Train_Game_01_00